In [ ]:
import numpy as np
from scipy.io import loadmat
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os
import warnings
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
output_notebook()

Determining N for the main script

In [ ]:
def determine_N(file_path, NFTP=1000, Gap=10, bin_width=250, max_dist=5000):
    # Determine_N
    # This will be a script that will allow you to determine "N" (i.e. when the
    # pairwise distance distributions reach a steady state) resulting in the
    # true pairwise distribution.
    # See "Determining the Frame Difference (N)" section of the USER GUIDE.
    
    # Load .mat or .npz data
    ext = os.path.splitext(file_path)[1].lower()
    
    if ext == '.mat':
        data = loadmat(file_path, verify_compressed_data_integrity=False, struct_as_record=False, squeeze_me=True)
        FI = data['Frame_Information']
        LF = data['LocalizationsFinal']
    elif ext == '.npz':
        npz = np.load(file_path, allow_pickle=True)
        FI = npz['Frame_Information'].tolist()
        LF = npz['LocalizationsFinal'].tolist()
    else:
        raise ValueError("Unsupported file format. Please provide a .mat or .npz file.")

    # Set up bins [0,250,500,...,5000,∞]
    bins = np.append(np.arange(0, max_dist + bin_width, bin_width), np.inf)

    Cum_Sum_Store = []
    Framestore = []

    print("Determining N")
    for iis in range(1, NFTP + 1, Gap):
        print(f"Fraction Done = {iis / NFTP:.3f}")
        total_blink = []

        for frames, coords in zip(FI, LF):
            if coords is None or len(coords) == 0:
                continue
            if len(coords) < 6000:
                fr = np.asarray(frames, dtype=float).reshape(-1, 1)
                Z2 = pdist(fr)                    # distances in frame-space
                D = pdist(np.asarray(coords))     # spatial distances
                mask = (np.abs(Z2 - iis) == 0)
                total_blink.append(D[mask])
        
        if total_blink:
            total_blink = np.hstack(total_blink)
        else:
            total_blink = np.array([])

        counts, _ = np.histogram(total_blink, bins=bins)
        if counts.sum() > 0:
            cdf = np.cumsum(counts / counts.sum())
        else:
            cdf = np.zeros_like(counts, dtype=float)

        Cum_Sum_Store.append(cdf)
        Framestore.append(iis)

    Cum_Sum_Store = np.vstack(Cum_Sum_Store)
    Z = np.sum(np.abs(Cum_Sum_Store - Cum_Sum_Store[0, :]), axis=1)

    print(Framestore)
    
    # Plotting
    plt.figure(figsize=(6, 4))
    plt.plot(Framestore, Z, linewidth=1.25)
    plt.xlabel('Frame')
    plt.ylabel('Z')
    plt.tight_layout()
    plt.savefig('temp_file_with_Z_vs_frame.png', dpi=300)
    plt.show()

    # Bokeh plot
    p = figure(title="Z vs Frame", width=600, height=400, x_axis_label='Frame', y_axis_label='Z')
    p.line(Framestore, Z, line_width=2, legend_label="Z", line_color="navy")
    hover = HoverTool(tooltips=[("Frame", "@x"), ("Z", "@y")])
    p.add_tools(hover)
    show(p)
    
    return Framestore, Z

Determining Res for the main script

In [8]:
def determine_ddc_resolution(file_path: str, N_f=200):
    """
    Determines the optimal bin resolution for DDC analysis from a .mat file.

    Parameters:
    - file_path: str, path to the .mat file
    - N_f: int, frame difference threshold for steady state

    Returns:
    - Optimal bin resolution for DDC.
    """
    data = loadmat(file_path, struct_as_record=False, squeeze_me=True)
    Frame_Information = data['Frame_Information']
    LocalizationsFinal = data['LocalizationsFinal']

    D_maxf = 0
    for loc in LocalizationsFinal:
        if loc is not None and len(loc) > 1:
            D = pdist(loc)
            D_maxf = max(D_maxf, np.max(D))

    for Resolution in range(150, 0, -10):
        bins = np.append(np.arange(0, D_maxf + Resolution, Resolution), np.inf)
        Total_Blink, Total_No_Blink = [], []

        for frames, coords in zip(Frame_Information, LocalizationsFinal):
            if coords is None or len(coords) < 2:
                continue

            Z2 = pdist(np.column_stack((np.zeros(len(frames)), frames)))
            D = pdist(coords)

            D_Blink = D[Z2 < N_f]
            D_No_Blink = D[Z2 > N_f]

            D_Counts = np.histogram(D_Blink, bins=bins, density=True)[0]
            D_Counts2 = np.histogram(D_No_Blink, bins=bins, density=True)[0]

            Total_Blink.append(D_Counts)
            Total_No_Blink.append(D_Counts2)

        D_Counts = np.mean(Total_Blink, axis=0)
        D_Counts2 = np.mean(Total_No_Blink, axis=0)

        lsne = next((i for i in range(1, len(D_Counts)) if i * Resolution > 1000), len(D_Counts))
        if np.sum(D_Counts2[lsne:]) == 0:
            continue
        D_Scale = np.sum(D_Counts[lsne:]) / np.sum(D_Counts2[lsne:])
        D_Counts3 = D_Counts - D_Counts2 * D_Scale
        D_Counts3[D_Counts3 < 0] = 0
        if np.sum(D_Counts3) == 0:
            continue
        D_Counts3 /= np.sum(D_Counts3)

        # Clean up to enforce decreasing behavior
        for i in range(3, len(D_Counts3) - 1):
            if D_Counts3[i + 1] >= D_Counts3[i]:
                D_Counts3[i] = 0
        D_Counts3[D_Counts3 < 0] = 0
        if np.sum(D_Counts3) == 0:
            continue
        D_Counts3 /= np.sum(D_Counts3)

        if np.sum(D_Counts3[8:] > 0) > 1:
            D_Counts3[8:] = 0
            D_Counts3 /= np.sum(D_Counts3)

        if np.argmax(D_Counts3) != 0:
            return Resolution + 10

    return 10

In [ ]:
# Use tkinter file dialog to mimic uigetfile
# This is useful to upload the mat file we need
Tk().withdraw()
filename_full = askopenfilename(filetypes=[("MAT or NPZ files", "*.mat *.npz")], title="Select .mat or .npz file")
if not filename_full:
    print('Error! No (or wrong) file selected!')
    exit()

# Call the function with the selected file
determine_N(filename_full, Gap=50)

In [ ]:
determine_ddc_resolution(filename_full, N_f=200)